In [1]:
import getpass
import os
os.environ["COHERE_API_KEY"] = getpass.getpass()

In [2]:
import json

with open("gods.json", "r", encoding="utf-8") as file:
    gods = json.load(file)
print('Total number of items:', len(gods))
print('First item:', gods[0])

Total number of items: 14
First item: {'name': 'زئوس', 'description': 'زئوس بر کوه الیمپوس حکومت می\u200cکند و پادشاه خدایان است. او پسر کرونوس و ریا، شوهر هرا است، و 15 کودک دارد که همه آنها المپیایی هستند.', 'appearance': 'او به عنوان یک مرد قد بلند و ستبر با لباس فاخر نشان داده شده است. او دارای موی بلند و ریش بلند است. به طور کلی، او به صورت ایستاده به تصویر درآمده است.', 'features': 'رعد، اسلحه او و ویژگی اصلی وی است که به طور کلی در دست راست او قرار دارد. گاهی اوقات او یک عصای سلطنتی را در دست چپ خود حمل می\u200cکند. گاو و عقاب حیوانات مقدس او هستند.'}


In [3]:
gods_text = ""
for god in gods:
    god_description = f"{god['name']}: {god['description']} {god['appearance']} {god['features']}"
    gods_text += god_description + "\n\n"

system_message = f"اطلاعات خدایان:\n\n{gods_text}\nپاسخ باید در حداکثر 4 الی 5 کلمه خلاصه شود!\nاگر نتوانستی در 4 الی 5 کلمه پاسخ بدی با رشته ی «نمی‌دانم» پاسخ بده.\nاگر پاسخ سوالی را نمیدانی با رشته ی «نمی‌دانم» پاسخ بده."

In [4]:
examples = [
    {"question": "کدام خدا آتش را به انسان‌ها داد؟", "answer": "پرومته"},
    {"question": "نام معبد آپولون کجاست؟", "answer": "دلفی"},
    {"question": "چه کسی پاندورا را ساخت؟", "answer": "هفایستوس"},
    {"question": "نام رود عالم زیرین چیست؟", "answer": "استیکس"},
    {"question": "چه کسی دیوارهای تروی را ساخت؟", "answer": "پوسوئیدون"},
    {"question": "کدام خدا هیولای پیتون را کشت؟", "answer": "آپولون"},
    {"question": "نام جزیره تولد آرتمیس چیست؟", "answer": "دلوس"},
    {"question": "چه کسی هلن را ربود؟", "answer": "پاریس"},
    {"question": "کدام خدا طوفان ادیسه را ایجاد کرد؟", "answer": "پوسوئیدون"},
    {"question": "چه کسی جعبه پاندورا را باز کرد؟", "answer": "اپیمته"},
    {"question": "نام هیولای سر شیر چیست؟", "answer": "کایمرا"},
    {"question": "چه کسی به عنکبوت تبدیل شد؟", "answer": "آراخنه"},
    {"question": "کدام خدا لابیرنت کرت را ساخت؟", "answer": "ددالوس"},
    {"question": "نام شهر حامی آتنا چیست؟", "answer": "آتن"},
    {"question": "چه کسی با بال‌های مومی پرواز کرد؟", "answer": "ایکاروس"},
    {"question": "کدام خدا تیتان‌ها را شکست داد؟", "answer": "زئوس"},
    {"question": "چه کسی مدوسا را نفرین کرد؟", "answer": "آتنا"},
    {"question": "نام شیر کشته‌شده توسط هراکلس چیست؟", "answer": "شیر نومه"},
    {"question": "کدام خدا نگهبان دروازه‌های المپوس بود؟", "answer": "هبه"},
    {"question": "چه کسی اشعار عاشقانه سرود؟", "answer": "آپولون"},
    {"question": "نام مسابقه انتخاب همسر هیپومنه چیست؟", "answer": "مسابقه دو"},
    {"question": "چه کسی سیب طلایی را اهدا کرد؟", "answer": "پاریس"},
    {"question": "کدام خدا به دلیل عشق به نرگس شناخته می‌شود؟", "answer": "نارسیس"},
    {"question": "چه کسی پارچه بی‌نقص بافت؟", "answer": "آتنا"},
    {"question": "کدام خدا ضیافت‌های المپوس را برگزار کرد؟", "answer": "دیونیسوس"},
    {"question": "چه کسی زئوس را با استخوان فریب داد؟", "answer": "پرومته"},
    {"question": "نام موجودی که هراکلس آن را شکار کرد چیست؟", "answer": "گراز اریانتیوس"},
    {"question": "کدام خدا ارواح را هدایت می‌کرد؟", "answer": "هرمس"},
    {"question": "چه کسی دیوارهای شهر آتن را تقویت کرد؟", "answer": "آتنا"},
    {"question": "نام موجودی که مینوتور را کشت چیست؟", "answer": "تسئوس"}
]

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model='embed-multilingual-v3.0')
to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [6]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=5,
)

example_selector.select_examples({"question": "در مورد زئوس بگو"})

[{'answer': 'زئوس', 'question': 'کدام خدا تیتان\u200cها را شکست داد؟'},
 {'answer': 'پرومته', 'question': 'چه کسی زئوس را با استخوان فریب داد؟'},
 {'answer': 'تسئوس', 'question': 'نام موجودی که مینوتور را کشت چیست؟'},
 {'answer': 'نارسیس',
  'question': 'کدام خدا به دلیل عشق به نرگس شناخته می\u200cشود؟'},
 {'answer': 'پوسوئیدون', 'question': 'کدام خدا طوفان ادیسه را ایجاد کرد؟'}]

In [7]:
from langchain_core.prompts import ChatPromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{answer}")]
    )
)

prompt = ChatPromptTemplate.from_messages(
    [
     ("system", system_message),
     few_shot_prompt,
     ("human", "{question}")
    ]
)

In [8]:
from langchain_cohere import ChatCohere

model = ChatCohere(top_p=0.9, tempreature=0)

In [9]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

In [15]:
question = input("سوال خود را بپرسید: ")
answer = chain.invoke(question)
print("پاسخ:", answer)

پاسخ: الهه حکمت و جنگ.
